## plotly implementation

In [29]:
(require '[clojupyter.misc.display :as display]
         '[clojure.java.io :as io]
         '[cheshire.core :as cheshire]
         '[clj-http.client :as client]
         '[clojure.repl :refer :all])  ;; to have all the repl goodies

To access plotly api, we need an api key.  The standard way is to store the api key in `$HOME/.plotly/.credentials` in a json object as defined in https://plot.ly/python/getting-started/.  For that you will need to create a plotly account.

In [61]:
(defn get-plotly-credentials
    "Returs the plotly credentials in $HOME/.plotly/.credentials"
    []
    (let [home (-> "HOME" System/getenv io/as-file)
          credential-file (io/file home ".plotly" ".credentials")]
        (if (.exists credential-file)
            (-> credential-file slurp (cheshire/parse-string true))
            {})))

(def plotly-cred  (get-plotly-credentials))
(println "your credentials are:\n" plotly-cred)

your credentials are:
 {:api_key 6FaFG4sgtpLnDrvzvFT1, :username didier, :stream_ids [8rcf9eam9j zupv6k3u4k]}


---

Next, we need to create "grids".  Grids are dataset stored on plotly that we can then plot.  The plotting is done in two phases, uploding the datasets and then plotting the datasets.

In [86]:
(defn upload-grid [grid cred]  
  (let [url  "https://api.plot.ly/v2/grids";;"https://api.plot.ly/v2/grids" "https://api.plot.ly/v2/grids" ;;
        headers {:Plotly-Client-Platform' "curl"}
        auth "didier:6FaFG4sgtpLnDrvzvFT1"];;[(:username cred) (:api_key cred)]]
    (println auth)
    (client/post url {:basic-auth auth
                      :headers headers
                      :throw-entire-message? true
                      :form-params grid :content-type :json})))

#'user/upload-grid

In [87]:
(def my-grid {:data 
              {:cols
               {
                   :height {:data [10 203 102 404] :order 1}
                   :weight {:data [112 234 321 34] :order 2}}}})
    

#'user/my-grid

In [87]:
(println (:body (upload-grid my-grid plotly-cred)))

didier:6FaFG4sgtpLnDrvzvFT1
ExceptionInfo clj-http: status 401 {:request-time 908, :repeatable? false, :protocol-version {:name "HTTP", :major 1, :minor 1}, :streaming? true, :chunked? true, :reason-phrase "UNAUTHORIZED", :headers {"Server" "nginx", "Via" "1.1 google", "Content-Type" "application/json", "Alt-Svc" "clear", "X-Frame-Options" "SAMEORIGIN", "Strict-Transport-Security" "max-age=31536000; includesubdomains; preload", "WWW-Authenticate" "Bearer realm=\"api\"", "Allow" "GET, POST, HEAD, OPTIONS", "Connection" "close", "Transfer-Encoding" "chunked", "Set-Cookie" ["plotly_sess_pr=z3l151u0weiecpy1jgmvp4bqaic9ksoo; Domain=.plot.ly; expires=Thu, 30-Nov-2017 03:09:02 GMT; httponly; Max-Age=1209600; Path=/; secure" "plotly_csrf_pr=b3rFF0ApmkYeHCK9Rjn9kKW30z2OyDkZ; Domain=.plot.ly; expires=Sat, 17-Nov-2018 03:09:02 GMT; Max-Age=31622400; Path=/; secure" "plotly_anoncsrf_pr=1Pv2vgnQIhIqOnw6bFy1kf4V2qAyXJ0r; Domain=.plot.ly; expires=Fri, 17-Nov-2017 03:09:02 GMT; httponly; Max-Age=86400

class clojure.lang.ExceptionInfo: 

curl --user didier:6FaFG4sgtpLnDrvzvFT1 -H "Content-Type: application/json" -H "Plotly-Client-Platform: curl" https://api.plot.ly/v2/grids --data '{"data": { "cols": { "first column": {"data": ["a", "b", "c"], "order": 0}, "second column": {"data": [1, 2, 3], "order": 1} } } }'